<a href="https://colab.research.google.com/github/quoctrungsz27/Week6_AI/blob/main/week6_MobileRobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy   # thư viện fuzzy logic  ( logic mờ )


In [ ]:
import numpy as np 
import skfuzzy as fuzz 
from skfuzzy import control as ctrl 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D

universe = np.linspace(18, 30,61)
universe1 = np.linspace(-5, 60, 61)
universe2 = np.linspace(0, 255, 61)

setpoint = ctrl.Antecedent(universe, 'setpoint')
sensor = ctrl.Antecedent(universe1, 'sensor')
pwm = ctrl.Consequent(universe2, 'pwm')

#names = [1, 2, 3, 4, 5]
names = ['VL','L','N','H','VH']
setpoint.automf(names = names)
sensor.automf(names = names)
pwm.automf(names = names) 

rule0 = ctrl.Rule(antecedent = ((setpoint['VL'] & sensor['VL'])|(setpoint['L'] & sensor['VL'])|(setpoint['VL'] & sensor['L'])),
                  consequent = pwm['VH'],label = 'ruleVL')
rule1 = ctrl.Rule(antecedent = ((setpoint['N'] & sensor['VL'])|(setpoint['L'] & sensor['L'])|(setpoint['VL'] & sensor['N'])),
                  consequent = pwm['H'],label = 'ruleL')
rule2 = ctrl.Rule(antecedent = ((setpoint['N'] & sensor['VL'])|(setpoint['N'] & sensor['L'])|(setpoint['L'] & sensor['N'])|(setpoint['VL'] & sensor['H'])|(setpoint['VH'] & sensor['VL'])|(setpoint['H'] & sensor['L'])|(setpoint['N'] & sensor['N'])|(setpoint['L'] & sensor['H'])|(setpoint['VL'] & sensor['VH'])),
                  consequent = pwm['N'],label = 'ruleN')
rule3 = ctrl.Rule(antecedent = ((setpoint['VH'] & sensor['L'])|(setpoint['H'] & sensor['N'])|(setpoint['N'] & sensor['H'])|(setpoint['L'] & sensor['VH'])|(setpoint['VH'] & sensor['N'])|(setpoint['H'] & sensor['H'])|(setpoint['N'] & sensor['VH'])),
                  consequent = pwm['L'],label = 'ruleH')
rule4 = ctrl.Rule(antecedent = ((setpoint['VH'] & sensor['H'])|(setpoint['H'] & sensor['VH'])|(setpoint['VH'] & sensor['VH'])),
                  consequent = pwm['VL'],label = 'ruleVH')
temp = ctrl.ControlSystem(rules =[rule0, rule1, rule2, rule3, rule4])
sim = ctrl.ControlSystemSimulation(temp,flush_after_run =61*61+1)
unsampled = np.linspace(18,30,61)
x,y = np.meshgrid(unsampled,unsampled)
z = np.zeros_like(x)

for i in range(61):
  for j in range(61): 
    sim.input['setpoint'] = x[i,j]
    sim.input['sensor'] = y[i,j]
    sim.compute() 
    z[i,j] = sim.output['pwm']

#ve bieu do
fig = plt.figure(figsize = (8,8))  # Hinh 8x8
ax = fig.add_subplot(111,projection='3d')
suf = ax.plot_surface(y,x,z,rstride = 1, cstride = 1, cmap = 'viridis', linewidth = 0.4, antialiased = True)
#cset = ax.contourf(x,y,z,zdir = 'z', offset = 8, cmap = 'viridis', anpha = 0.5)
#cset = ax.contourf(x,y,z,zdir = 'x', offset = 11, cmap = 'viridis', anpha = 0.5)
#cset = ax.contourf(x,y,z,zdir = 'y', offset = 11, cmap = 'viridis', anpha = 0.5)
ax.view_init(30,200)




